# **Procesamiento de Informacion Manipulada**

En esta parte del proyecto se procesara las transcripciones obtenidas, para analizar y detectar:

- Groserias
- Mensajes de odio
- Analisis de sentimientos
- Análisis de emociones
- Menciones

# Pre-procesamiento y Limpieza

### Instalacion de librerias

In [ ]:
# instalacion de la libreria
!pip install spanlp # para deteccion de groserias y limpieza de datos

  Using cached spanlp-1.1.0-py3-none-any.whl.metadata (25 kB)
Using cached spanlp-1.1.0-py3-none-any.whl (40 kB)


### Importacionde de librerias pre-procesamiento

In [ ]:
import pandas as pd
from google.colab import drive

# spanlp para la limpieza de datos
from spanlp.domain.strategies import Preprocessing, TextToLower, RemoveUserMentions, RemovePunctuation, RemoveUnicodeCharacters, RemoveUrls, RemoveTicks, RemoveBackTicks, RemoveAccents, RemoveStopWords, RemoveArticles, RemoveConjunctions, RemoveHtmlTags, RemoveEmailAddress, ExpandAbbreviations, RemoveAccents, RemoveEmoticons, RemoveHashtags, RemoveExtraSpaces, RemoveUrls

# Gensim para tokenizacion
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora

### Conexion con google drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Importacion de las transcripciones

In [ ]:
df_orig = pd.read_excel('/content/drive/MyDrive/Proyecto grado/Transcripciones definitivas/Transcripciones (1) (1).xlsx')
df_orig.sample(5)

In [ ]:
# metadatos
df_orig.info()

### Limpieza de datos

Datos nulos o sin transcripcion

In [ ]:
df_orig.isna().sum()

NameError: name 'df_orig' is not defined

In [ ]:
df_nulos = df_orig[df_orig.isna().any(axis=1)]
df_nulos

In [ ]:
# eliminar filas nulas
df_clean = df_orig.dropna()

# verificar filas nulas
df_clean.isna().sum()

In [ ]:
df_clean.info()

### Limpieza de las transcripciones

In [ ]:
df_clean = df_clean.copy()

# Creamos una nueva columna donde se almacenara el texto procesado
df_clean['trans_clean'] = ''

Usaremos la libreria de Preprocessing para la limpieza de las transcripciones.


Estas son las estrategias de limpieza empleadas:

- **TextToLower** input: "HOLA QUE MÁS?" output: "hola que más?"
- **RemoveUserMentions** input: "Hola @channel como van?" output: "Hola como van?"
- **RemovePunctuation** input: "Mensaje...,con, puntos" output: "Mensaje con puntos"
- **RemoveUnicodeCharacters** input: "hola çcomo ªvan?" output: "hola como van" (Unicode characters)
- **RemoveUrls** input: "Revisen este recurso https://github.com/jfreddypuentes/spanlp" output: "Revisen este recurso "
- **RemoveTicks** input: "Hola, que' más'" output: "Hola, que más"
- **RemoveBackTicks** input: "Hola, que más" output: "Hola, que más"
- **RemoveAccents** input: "La canción es una sensación" output: "La cancion es una sensacion"
- **RemoveStopWords** input: "La canción y la letra es buena" output: "canción letra buena"
- **RemoveArticles** input: "La canción y la letra es buena" output: "canción y letra es buena"
- **RemoveConjunctions** input: "y entonces estaba programando aunque con sueño pero concentrado creando esta libreria" output: "entonces estaba programando con sueño concentrado creando esta libreria"
- **RemoveHtmlTags** input: "Hola USUARIO que tal?" output: "Hola USUARIO que tal?"
- **RemoveEmailAddress** input: "Hola Pepito, el correo es contacto@domain.com" output: "Hola Pepito, el correo es "
- **ExpandAbbreviations** input: "pero xq tengo es3 si yo estaba bn en clase, ahora me duelen to2 los musculos" output: "pero por que tengo estres si yo estaba bien en clase, ahora me duelen todos los musculos"
- **RemoveAccents** input: "La canción es una sensación" output: "La cancion es una sensacion"

In [ ]:
strategies = [TextToLower(), RemoveUserMentions(), RemovePunctuation(),RemoveAccents(), RemoveUrls(), RemoveStopWords(), RemoveArticles(), RemoveConjunctions(), RemoveHtmlTags(), RemoveEmailAddress(), ExpandAbbreviations(), RemoveTicks (), RemoveBackTicks()]

In [ ]:
for index, row in df_clean.iterrows():

  trans_actual=row.transcripcion

  try:
    cleaned = Preprocessing().clean(data=trans_actual, clean_strategies=strategies)
    df_clean.at[index, 'trans_clean'] = cleaned
  except:
    continue

In [ ]:
df_clean

### Tokenizacion
Agregamos una nueva columna con la tokenizacion de las transcripciones para su posterior procesamiento.

In [ ]:
# nueva columna para la tokenizacion
df_clean['token_text'] = ''

In [ ]:
# funcion para tokenizar texto
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(token)

    return result

In [ ]:
# Aplicamos la funcion al df
df_clean['token_text'] = df_clean['trans_clean'].apply(preprocess)

In [ ]:
df_clean

# Detección de Groserias

**Spanlp**

spanlp es una librería escrita en Python para detección de groserías, vulgaridades, palabras de odio, racismo, xenofobia y bullying en textos escritos en Español.

repositorio de la libreria y mas informacion: https://github.com/jfreddypuentes/spanlp

In [ ]:
# importacion de la libreria
from spanlp.palabrota import Palabrota
from spanlp.domain.countries import Country

### Diccionario especifico para palabras de Colombia

Creamos dos nuevas columnas, una para determinar por un boolean si hay alguna groseria "**Groserias_colombia**" y la otra para poner la o las groserias detectadas "**Malas_Palabras_colombia**".

Esta primera iteracion solo se usara con palabras especificamente para colombia


In [ ]:
df_clean["Groserias_colombia"]= " "
df_clean["Malas_Palabras_colombia"]=" "

In [ ]:
palabrota = Palabrota(countries=[Country.COLOMBIA]) # funcion para detectar groserias

for index, row in df_clean.iterrows():

  trans_array = row.token_text

  try:

    es_groseria = False
    malas_p = ""

    for p in trans_array:

      contiene_palabrota = palabrota.contains_palabrota(p)

      if contiene_palabrota:
        malas_p += p + ", "
        es_groseria = True

    if es_groseria:
      malas_p = malas_p[:-2] #limpia la ","

    df_clean.at[index, "Groserias_colombia"] = es_groseria
    df_clean.at[index, "Malas_Palabras_colombia"] = malas_p

  except:
    continue

In [ ]:
df_clean

,id,page_name,transcripcion,trans_clean,token_text,Groserias_colombia,Malas_Palabras_colombia
0,305456388760154,Juan_David_Valderrama,"Vamos a crear el Team Medellín, una gran apues...",vamos crear team medellin gran apuesta gente j...,"[vamos, crear, team, medellin, gran, apuesta, ...",,
1,331829125877865,Juan_David_Valderrama,"Vamos a crear el Team Medellín, una gran apues...",vamos crear team medellin gran apuesta gente j...,"[vamos, crear, team, medellin, gran, apuesta, ...",,
2,356049983512691,Juan_David_Valderrama,"Vamos a crear el Team Medellín, una gran apues...",vamos crear team medellin gran apuesta gente j...,"[vamos, crear, team, medellin, gran, apuesta, ...",,
3,684977159916656,Juan_David_Valderrama,le saca la piedra? La injusticia y las mentira...,saca piedra injusticia mentiras ¿y hace cagars...,"[saca, piedra, injusticia, mentiras, hace, cag...",,
4,1345513279656989,Juan_David_Valderrama,le saca la piedra? La injusticia y las mentira...,saca piedra injusticia mentiras ¿y hace cagars...,"[saca, piedra, injusticia, mentiras, hace, cag...",,
...,...,...,...,...,...,...,...
1738,306371302164384,Albert_Corredor,Aquí empezó todo. Estamos en el barrio que me ...,aqui empezo barrio vio crecer belen buenavista...,"[aqui, empezo, barrio, crecer, belen, buenavis...",,
1739,223787314050706,Albert_Corredor,"Porque este domingo familia, con la ayuda de D...",domingo familia ayuda dios belen buenavista po...,"[domingo, familia, ayuda, dios, belen, buenavi...",,
1740,703418311719188,Albert_Corredor,"Porque este domingo, familia, con la ayuda de ...",domingo familia ayuda dios belen buenavista po...,"[domingo, familia, ayuda, dios, belen, buenavi...",,
1741,158591357281269,Albert_Corredor,Hoy teníamos programado un evento con cerca de...,hoy programado evento 10000 personas ustedes s...,"[programado, evento, personas, ustedes, saben,...",,


In [ ]:
# convert to csv
df_clean.to_csv('df_clean.csv')

### Diccionario con todas las palabras latinoamericanas

Creamos dos nuevas columnas, una para determinar por un boolean si hay alguna groseria "Groserias_latinoamerica" y la otra para poner la o las groserias detectadas "Malas_Palabras_latinoamerica".

Esta segunda iteracion usara los diccionarios de todos los paises de Latinoamerica

In [ ]:
df_clean["Groserias_latinoamerica"]= " "
df_clean["Malas_Palabras_latinoamerica"]=" "

NameError: name 'df_clean' is not defined

In [ ]:
palabrota = Palabrota() # funcion para detectar groserias

for index, row in df_clean.iterrows():

  trans_array = row.token_text

  try:

    es_groseria = False
    malas_p = ""

    for p in trans_array:

      contiene_palabrota = palabrota.contains_palabrota(p)

      if contiene_palabrota:
        malas_p += p + ", "
        es_groseria = True

    if es_groseria:
      malas_p = malas_p[:-2] #limpia la ","

    df_clean.at[index, "Groserias_latinoamerica"] = es_groseria
    df_clean.at[index, "Malas_Palabras_latinoamerica"] = malas_p

  except:
    continue

In [ ]:
df_clean

In [ ]:
# convert to csv
df_clean.to_csv('df_clean.csv')

### Iteracion con diccionario de palabras colombianas coloquiales

Creamos dos nuevas columnas, una para determinar por un boolean si hay alguna groseria "Groserias_colombia2" y la otra para poner la o las groserias detectadas "Malas_Palabras_colombia2".

Esta tercera iteracion usara un diccionario de palabras tomado por *Nancy Rozo Instituto Caro y Cuervo Colombia*

Fuente de palabras original: https://github.com/sgjimenezv/spanish_regional_words_benchmark/blob/master/diccionarios.py


In [ ]:
df_clean["Groserias_colombia2"]= " "
df_clean["Malas_Palabras_colombia2"]=" "

Importacion del conjunto de palabras

In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/Proyecto grado/Codigo/')
from dic_colombia import ColombiaVocabulary

In [ ]:
vocabulary = ColombiaVocabulary() #instanciamos la clase
words = vocabulary.get_words()
print(words)

In [ ]:
for index, row in df_clean.iterrows():

  trans_array = row.token_text

  try:

    es_groseria = False
    malas_p = ""

    for p in trans_array:

      contiene_palabrota = vocabulary.contains(p)

      if contiene_palabrota:
        malas_p += p + ", "
        es_groseria = True

    if es_groseria:
      malas_p = malas_p[:-2] #limpia la ","

    df_clean.at[index, "Groserias_colombia2"] = es_groseria
    df_clean.at[index, "Malas_Palabras_colombia2"] = malas_p

  except:
    continue

In [ ]:
df_clean

In [ ]:
# convert to csv
df_clean.to_csv('df_clean_g.csv')

NameError: name 'df_clean' is not defined

# Detección de discurso de odio

*   Elemento de lista

> Añadir blockquote


*   Elemento de lista



In [ ]:
# carga del ultimo df guardado
df_clean = pd.read_csv('/content/drive/MyDrive/Proyecto grado/Codigo/df_clean_d2 - adcreative.csv')

In [ ]:
df_clean

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,page_name,ad_creative_bodies,trans_clean,token_text,Groserias_colombia,Lenguaje informal,Groserias_latinoamerica,Malas_Palabras_latinoamerica,PREDICCIONES,sentimiento,prob_sentimiento,emocion,prob_emocion,dic_candi_presid,dic_partidos
0,0,0,0,0,1.435098e+15,Luis Bernardo Vélez,['Madrugo a trabajar por la causa Medellín. Le...,madrugo trabajar causa medellin entrego experi...,"['madrugo', 'trabajar', 'causa', 'medellin', '...",False,NaN,False,NaN,0.0,POS,"{'NEG': 0.03657609224319458, 'NEU': 0.40292927...",joy,"{'others': 0.18212905526161194, 'joy': 0.81201...","['luis', 'bernardo']",[]
1,1,1,1,1,8.087715e+14,Luis Bernardo Vélez,['Gracias a Medellín. Gracias a tantas persona...,gracias medellin gracias tantas personas palab...,"['gracias', 'medellin', 'gracias', 'tantas', '...",False,NaN,False,NaN,0.0,POS,"{'NEG': 0.01415505912154913, 'NEU': 0.14502669...",joy,"{'others': 0.019396435469388962, 'joy': 0.9780...","['luis', 'bernardo']",[]
2,2,2,2,2,8.712091e+14,Luis Bernardo Vélez,"['Soy Luis Bernardo Vélez, quiero contarles de...",luis bernardo velez quiero contarles vengo voy...,"['luis', 'bernardo', 'velez', 'quiero', 'conta...",True,NaN,True,NaN,0.0,NEU,"{'NEG': 0.02514345571398735, 'NEU': 0.62056976...",others,"{'others': 0.9425402283668518, 'joy': 0.052007...",[],[]
3,3,3,3,3,1.700861e+15,Luis Bernardo Vélez,"['Soy Luis Bernardo Vélez, quiero contarles de...",luis bernardo velez quiero contarles vengo voy...,"['luis', 'bernardo', 'velez', 'quiero', 'conta...",True,NaN,True,NaN,0.0,NEU,"{'NEG': 0.03485953062772751, 'NEU': 0.69108456...",others,"{'others': 0.9509748220443726, 'joy': 0.045762...","['luis', 'bernardo']",[]
4,4,4,4,4,3.372506e+14,Luis Bernardo Vélez,"['Nos han invisibilizado, han vulnerado nuestr...",invisibilizado vulnerado derechos candidatos h...,"['invisibilizado', 'vulnerado', 'derechos', 'c...",True,NaN,True,NaN,0.0,NEG,"{'NEG': 0.6510934829711914, 'NEU': 0.242750272...",others,"{'others': 0.5393030643463135, 'joy': 0.041482...",[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,1796,1796,1796,1863,8.023435e+14,Felipe Vélez,['Medellín necesita nuevos liderazgos. Necesit...,medellin necesita nuevos liderazgos necesitamo...,"['medellin', 'necesita', 'nuevos', 'liderazgos...",False,NaN,False,NaN,0.0,NEU,"{'NEG': 0.08532607555389404, 'NEU': 0.68700987...",others,"{'others': 0.9694794416427612, 'joy': 0.023887...",[],['nuevo']
1797,1797,1797,1797,1864,7.224310e+14,Carlos Alberto Ballesteros Baron,['Este 29 de octubre Votamos #CarlosBallestero...,29 octubre votamos carlosballesterosalcalde ad...,"['octubre', 'votamos', 'administracion', 'incl...",False,NaN,False,NaN,0.0,NEU,"{'NEG': 0.0735396295785904, 'NEU': 0.648458003...",others,"{'others': 0.637188732624054, 'joy': 0.3568516...",[],"['polo', 'democratico']"
1798,1798,1798,1798,1865,8.447881e+14,Carlos Alberto Ballesteros Baron,['#PorFinUnaMedellínDeTodxs | Cada vez somos m...,porfinunamedellindetodxs cada vez vamos trans...,"['cada', 'vamos', 'transformar', 'ciudad', 'ma...",True,NaN,True,NaN,0.0,POS,"{'NEG': 0.028913510963320732, 'NEU': 0.3447154...",joy,"{'others': 0.06268776953220367, 'joy': 0.93330...",[],['somos']
1799,1799,1799,1799,1866,1.047580e+15,Carlos Alberto Ballesteros Baron,['#Medellín| Nuestra visión de la relación ent...,medellin vision relacion actores publicos priv...,"['medellin', 'vision', 'relacion', 'actores', ...",True,NaN,True,NaN,0.0,NEU,"{'NEG': 0.1337633579969406, 'NEU': 0.664075911...",others,"{'others': 0.9928483963012695, 'joy': 0.004709...",[],[]


In [ ]:
import ast
df_clean['ad_creative_bodies_text'] = df_clean['ad_creative_bodies'].apply(lambda x: ast.literal_eval(x)[0] if isinstance(x, str) else x)

In [ ]:
df_clean

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,page_name,ad_creative_bodies,trans_clean,token_text,Groserias_colombia,...,Groserias_latinoamerica,Malas_Palabras_latinoamerica,PREDICCIONES,sentimiento,prob_sentimiento,emocion,prob_emocion,dic_candi_presid,dic_partidos,ad_creative_bodies_text
0,0,0,0,0,1.435098e+15,Luis Bernardo Vélez,['Madrugo a trabajar por la causa Medellín. Le...,madrugo trabajar causa medellin entrego experi...,"['madrugo', 'trabajar', 'causa', 'medellin', '...",False,...,False,NaN,0.0,POS,"{'NEG': 0.03657609224319458, 'NEU': 0.40292927...",joy,"{'others': 0.18212905526161194, 'joy': 0.81201...","['luis', 'bernardo']",[],Madrugo a trabajar por la causa Medellín. Le e...
1,1,1,1,1,8.087715e+14,Luis Bernardo Vélez,['Gracias a Medellín. Gracias a tantas persona...,gracias medellin gracias tantas personas palab...,"['gracias', 'medellin', 'gracias', 'tantas', '...",False,...,False,NaN,0.0,POS,"{'NEG': 0.01415505912154913, 'NEU': 0.14502669...",joy,"{'others': 0.019396435469388962, 'joy': 0.9780...","['luis', 'bernardo']",[],Gracias a Medellín. Gracias a tantas personas ...
2,2,2,2,2,8.712091e+14,Luis Bernardo Vélez,"['Soy Luis Bernardo Vélez, quiero contarles de...",luis bernardo velez quiero contarles vengo voy...,"['luis', 'bernardo', 'velez', 'quiero', 'conta...",True,...,True,NaN,0.0,NEU,"{'NEG': 0.02514345571398735, 'NEU': 0.62056976...",others,"{'others': 0.9425402283668518, 'joy': 0.052007...",[],[],"Soy Luis Bernardo Vélez, quiero contarles de d..."
3,3,3,3,3,1.700861e+15,Luis Bernardo Vélez,"['Soy Luis Bernardo Vélez, quiero contarles de...",luis bernardo velez quiero contarles vengo voy...,"['luis', 'bernardo', 'velez', 'quiero', 'conta...",True,...,True,NaN,0.0,NEU,"{'NEG': 0.03485953062772751, 'NEU': 0.69108456...",others,"{'others': 0.9509748220443726, 'joy': 0.045762...","['luis', 'bernardo']",[],"Soy Luis Bernardo Vélez, quiero contarles de d..."
4,4,4,4,4,3.372506e+14,Luis Bernardo Vélez,"['Nos han invisibilizado, han vulnerado nuestr...",invisibilizado vulnerado derechos candidatos h...,"['invisibilizado', 'vulnerado', 'derechos', 'c...",True,...,True,NaN,0.0,NEG,"{'NEG': 0.6510934829711914, 'NEU': 0.242750272...",others,"{'others': 0.5393030643463135, 'joy': 0.041482...",[],[],"Nos han invisibilizado, han vulnerado nuestros..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,1796,1796,1796,1863,8.023435e+14,Felipe Vélez,['Medellín necesita nuevos liderazgos. Necesit...,medellin necesita nuevos liderazgos necesitamo...,"['medellin', 'necesita', 'nuevos', 'liderazgos...",False,...,False,NaN,0.0,NEU,"{'NEG': 0.08532607555389404, 'NEU': 0.68700987...",others,"{'others': 0.9694794416427612, 'joy': 0.023887...",[],['nuevo'],Medellín necesita nuevos liderazgos. Necesitam...
1797,1797,1797,1797,1864,7.224310e+14,Carlos Alberto Ballesteros Baron,['Este 29 de octubre Votamos #CarlosBallestero...,29 octubre votamos carlosballesterosalcalde ad...,"['octubre', 'votamos', 'administracion', 'incl...",False,...,False,NaN,0.0,NEU,"{'NEG': 0.0735396295785904, 'NEU': 0.648458003...",others,"{'others': 0.637188732624054, 'joy': 0.3568516...",[],"['polo', 'democratico']",Este 29 de octubre Votamos #CarlosBallesterosA...
1798,1798,1798,1798,1865,8.447881e+14,Carlos Alberto Ballesteros Baron,['#PorFinUnaMedellínDeTodxs | Cada vez somos m...,porfinunamedellindetodxs cada vez vamos trans...,"['cada', 'vamos', 'transformar', 'ciudad', 'ma...",True,...,True,NaN,0.0,POS,"{'NEG': 0.028913510963320732, 'NEU': 0.3447154...",joy,"{'others': 0.06268776953220367, 'joy': 0.93330...",[],['somos'],#PorFinUnaMedellínDeTodxs | Cada vez somos más...
1799,1799,1799,1799,1866,1.047580e+15,Carlos Alberto Ballesteros Baron,['#Medellín| Nuestra visión de la relación ent...,medellin vision relacion actores publicos priv...,"['medellin', 'vision', 'relacion', 'actores', ...",True,...,True,NaN,0.0,NEU,"{'NEG': 0.1337633579969406, 'NEU': 0.664075911...",others,"{'others': 0.9928483963012695

In [ ]:
# limpieza de la nueva columna "ad_creative_bodies_text" para procesar los datos

#strategies = [RemoveEmoticons(),TextToLower(), RemoveUserMentions(), RemoveHtmlTags(), RemoveEmailAddress(), RemoveTicks(), RemoveBackTicks(), RemoveHashtags(),RemoveUnicodeCharacters(),RemoveExtraSpaces()]

strategies = [RemoveUrls()]

for index, row in df_clean.iterrows():

  trans_actual=row.ad_creative_bodies_text

  try:
    cleaned = Preprocessing().clean(data=trans_actual, clean_strategies=strategies)
    df_clean.at[index, 'ad_creative_bodies_text_clean'] = cleaned
  except:
    continue

In [ ]:
df_clean.isna().sum()

,0
Unnamed: 0.3,0
Unnamed: 0.2,0
Unnamed: 0.1,0
Unnamed: 0,0
id,0
page_name,0
ad_creative_bodies,0
trans_clean,10
token_text,0
Groserias_colombia,0


En esta parte del proyecto se utlizara un modelo ya entrenado para la deteccion de discursos de odio.

Repositorio original: https://github.com/esaidh266/Algorithm-for-detection-of-hate-speech-in-Spanish/tree/main

### Instalacion e implementacion del algoritmo y el modelo

In [ ]:
# llegar a al ubicacion donde se almacenara el codigo
%cd /content/drive/MyDrive/Proyecto\ grado/Codigo

/content/drive/MyDrive/Proyecto grado/Codigo


In [ ]:
# clonar repositorio
!git clone https://github.com/esaidh266/Algorithm-for-detection-of-hate-speech-in-Spanish.git

fatal: destination path 'Algorithm-for-detection-of-hate-speech-in-Spanish' already exists and is not an empty directory.


In [ ]:
# instalacion de dependencias
!pip install --upgrade requests urllib3 charset_normalizer chardet

In [ ]:
cd Algorithm-for-detection-of-hate-speech-in-Spanish/

/content/drive/MyDrive/Proyecto grado/Codigo/Algorithm-for-detection-of-hate-speech-in-Spanish


In [ ]:
ls

'=2.5.0'               df_merge_hs.csv                  __pycache__/
'=2.8.2'              'DOCUMENTACIÓN GITHUB.docx'       README.md
 Comparativa_V2.xlsx  'ejemplo (1).py'                  recursos/
'Demo video'           Licence                         'Recursos-20231027T110710Z-001 (1).zip'
 df_clean_26Feb.csv   'Modelo_binario_ (1) (1).ipynb'   Requirements.txt
 df_clean_g_p.csv      MODELOS/                         resultados_prePred.csv
 df_limpio.csv         obtener_caracteristicas.py


In [ ]:
# instalacion de depencias del requeriments.txt
!pip install -r Requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 45.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 67.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.0/170.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 10.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of datasets to determine which versi

In [ ]:
# conflicto de dependencias, se reinstala las libreria
!pip uninstall transformers torch tokenizers huggingface-hub -y
!pip install transformers torch

Found existing installation: transformers 4.40.1
Uninstalling transformers-4.40.1:
  Successfully uninstalled transformers-4.40.1
Found existing installation: torch 2.5.1+cu124
Uninstalling torch-2.5.1+cu124:
  Successfully uninstalled torch-2.5.1+cu124
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Found existing installation: huggingface-hub 0.28.1
Uninstalling huggingface-hub-0.28.1:
  Successfully uninstalled huggingface-hub-0.28.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.1/150.1 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.2/253.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.0/468.0 kB 28.9 MB/s eta 0:00:00
   

In [ ]:
# actualizacion de dependencias
!pip install --upgrade torch
!pip install --upgrade transformers tokenizers huggingface-hub
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 7.6 MB/s eta 0:00:00


In [ ]:
pip install torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.5/906.5 MB 703.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.3 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: torch
    Found existing installation: torch 2.6.0
    Uninstalling torch-2.6.0:
      Successfully uninstalled torch-2.6.0


### importacion de librerias

In [ ]:
!ls

'=2.5.0'	       df_merge_hs.csv		        __pycache__
'=2.8.2'	      'DOCUMENTACIÓN GITHUB.docx'       README.md
 Comparativa_V2.xlsx  'ejemplo (1).py'		        recursos
'Demo video'	       Licence			       'Recursos-20231027T110710Z-001 (1).zip'
 df_clean_26Feb.csv   'Modelo_binario_ (1) (1).ipynb'   Requirements.txt
 df_clean_g_p.csv      MODELOS			        resultados_prePred.csv
 df_limpio.csv	       obtener_caracteristicas.py


In [ ]:
# recargar modulo

import sys
import importlib

# Nombre del módulo que deseas eliminar y recargar
modulo = "obtener_caracteristicas"

# Eliminar el módulo si ya está cargado
if modulo in sys.modules:
    del sys.modules[modulo]

# Volver a importarlo
import obtener_caracteristicas
importlib.reload(obtener_caracteristicas)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'obtener_caracteristicas' from '/content/drive/MyDrive/Proyecto grado/Codigo/Algorithm-for-detection-of-hate-speech-in-Spanish/obtener_caracteristicas.py'>

In [ ]:
import joblib
import obtener_caracteristicas

### Carga del modelo

In [ ]:
modelo = joblib.load("/content/drive/MyDrive/Proyecto grado/Codigo/Algorithm-for-detection-of-hate-speech-in-Spanish/MODELOS/80-20/RF_binario_80-20.joblib")

In [ ]:
# renombramos la columna para ser analizada y copiamos el df para limpiarlo

df_clean = df_clean.rename(columns={'ad_creative_bodies_text_clean': 'CONTENIDO A ANALIZAR'})
#df_clean = df_clean.rename(columns={'trans_clean': 'CONTENIDO A ANALIZAR'})
#df_clean = df_clean.rename(columns={'ad_creative_bodies_text': 'CONTENIDO A ANALIZAR'})


In [ ]:
# Dataframe en el que mostrar los resultados
resultados = pd.DataFrame()
resultados['CONTENIDO A ANALIZAR'] = df_clean['CONTENIDO A ANALIZAR']

In [ ]:
resultados.to_csv('resultados_prePred.csv')

In [ ]:
df_clean.to_csv('df_clean_2Marz.csv')

In [ ]:
#df_clean.to_csv('df_clean_26Feb.csv')
df_clean

,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,id,page_name,ad_creative_bodies,trans_clean,token_text,Groserias_colombia,...,Malas_Palabras_latinoamerica,PREDICCIONES,sentimiento,prob_sentimiento,emocion,prob_emocion,dic_candi_presid,dic_partidos,ad_creative_bodies_text,CONTENIDO A ANALIZAR
0,0,0,0,0,1.435098e+15,Luis Bernardo Vélez,['Madrugo a trabajar por la causa Medellín. Le...,madrugo trabajar causa medellin entrego experi...,"['madrugo', 'trabajar', 'causa', 'medellin', '...",False,...,NaN,0.0,POS,"{'NEG': 0.03657609224319458, 'NEU': 0.40292927...",joy,"{'others': 0.18212905526161194, 'joy': 0.81201...","['luis', 'bernardo']",[],Madrugo a trabajar por la causa Medellín. Le e...,Madrugo a trabajar por la causa Medellín. Le e...
1,1,1,1,1,8.087715e+14,Luis Bernardo Vélez,['Gracias a Medellín. Gracias a tantas persona...,gracias medellin gracias tantas personas palab...,"['gracias', 'medellin', 'gracias', 'tantas', '...",False,...,NaN,0.0,POS,"{'NEG': 0.01415505912154913, 'NEU': 0.14502669...",joy,"{'others': 0.019396435469388962, 'joy': 0.9780...","['luis', 'bernardo']",[],Gracias a Medellín. Gracias a tantas personas ...,Gracias a Medellín. Gracias a tantas personas ...
2,2,2,2,2,8.712091e+14,Luis Bernardo Vélez,"['Soy Luis Bernardo Vélez, quiero contarles de...",luis bernardo velez quiero contarles vengo voy...,"['luis', 'bernardo', 'velez', 'quiero', 'conta...",True,...,NaN,0.0,NEU,"{'NEG': 0.02514345571398735, 'NEU': 0.62056976...",others,"{'others': 0.9425402283668518, 'joy': 0.052007...",[],[],"Soy Luis Bernardo Vélez, quiero contarles de d...","Soy Luis Bernardo Vélez, quiero contarles de d..."
3,3,3,3,3,1.700861e+15,Luis Bernardo Vélez,"['Soy Luis Bernardo Vélez, quiero contarles de...",luis bernardo velez quiero contarles vengo voy...,"['luis', 'bernardo', 'velez', 'quiero', 'conta...",True,...,NaN,0.0,NEU,"{'NEG': 0.03485953062772751, 'NEU': 0.69108456...",others,"{'others': 0.9509748220443726, 'joy': 0.045762...","['luis', 'bernardo']",[],"Soy Luis Bernardo Vélez, quiero contarles de d...","Soy Luis Bernardo Vélez, quiero contarles de d..."
4,4,4,4,4,3.372506e+14,Luis Bernardo Vélez,"['Nos han invisibilizado, han vulnerado nuestr...",invisibilizado vulnerado derechos candidatos h...,"['invisibilizado', 'vulnerado', 'derechos', 'c...",True,...,NaN,0.0,NEG,"{'NEG': 0.6510934829711914, 'NEU': 0.242750272...",others,"{'others': 0.5393030643463135, 'joy': 0.041482...",[],[],"Nos han invisibilizado, han vulnerado nuestros...","Nos han invisibilizado, han vulnerado nuestros..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1796,1796,1796,1796,1863,8.023435e+14,Felipe Vélez,['Medellín necesita nuevos liderazgos. Necesit...,medellin necesita nuevos liderazgos necesitamo...,"['medellin', 'necesita', 'nuevos', 'liderazgos...",False,...,NaN,0.0,NEU,"{'NEG': 0.08532607555389404, 'NEU': 0.68700987...",others,"{'others': 0.9694794416427612, 'joy': 0.023887...",[],['nuevo'],Medellín necesita nuevos liderazgos. Necesitam...,Medellín necesita nuevos liderazgos. Necesitam...
1797,1797,1797,1797,1864,7.224310e+14,Carlos Alberto Ballesteros Baron,['Este 29 de octubre Votamos #CarlosBallestero...,29 octubre votamos carlosballesterosalcalde ad...,"['octubre', 'votamos', 'administracion', 'incl...",False,...,NaN,0.0,NEU,"{'NEG': 0.0735396295785904, 'NEU': 0.648458003...",others,"{'others': 0.637188732624054, 'joy': 0.3568516...",[],"['polo', 'democratico']",Este 29 de octubre Votamos #CarlosBallesterosA...,Este 29 de octubre Votamos #CarlosBallesterosA...
1798,1798,1798,1798,1865,8.447881e+14,Carlos Alberto Ballesteros Baron,['#PorFinUnaMedellínDeTodxs | Cada vez somos m...,porfinunamedellindetodxs cada vez vamos trans...,"['cada', 'vamos', 'transformar', 'ciudad', 'ma...",True,...,NaN,0.0,POS,"{'NEG': 0.028913510963320732, 'NEU': 0.3447154...",joy,"{'others': 0.06268776953220367, 'joy': 0.93330...",[],['somos'],#PorFinUnaMedellínDeTodxs | Cada vez somos más...,#PorFinUnaMedellínDeTodxs | C

In [ ]:
# Procesamos el dataframe
df_clean = obtener_caracteristicas.main(df_clean)


100%|██████████| 1801/1801 [00:00<00:00, 13933.69it/s]


#################
[]
#################



100%|██████████| 1801/1801 [00:00<00:00, 3527.65it/s]

0it [00:00, ?it/s]
1it [00:00,  3.37it/s]
2it [00:00,  4.20it/s]
3it [00:00,  4.92it/s]
4it [00:00,  5.22it/s]
5it [00:01,  3.95it/s]
6it [00:01,  4.44it/s]
7it [00:01,  4.23it/s]
8it [00:01,  3.60it/s]
9it [00:02,  2.43it/s]
10it [00:03,  2.28it/s]
11it [00:03,  2.37it/s]
12it [00:03,  2.76it/s]
13it [00:04,  2.87it/s]
14it [00:04,  3.48it/s]
15it [00:04,  3.23it/s]
16it [00:04,  3.24it/s]
17it [00:05,  3.20it/s]
18it [00:05,  2.70it/s]
19it [00:06,  2.70it/s]
20it [00:06,  3.25it/s]
21it [00:06,  3.73it/s]
22it [00:06,  4.23it/s]
23it [00:06,  3.64it/s]
24it [00:07,  2.93it/s]
25it [00:07,  2.59it/s]
26it [00:08,  2.10it/s]
27it [00:09,  1.97it/s]
28it [00:09,  2.00it/s]
29it [00:10,  2.01it/s]
30it [00:10,  1.83it/s]
31it [00:11,  1.86it/s]
32it [00:11,  1.88it/s]
33it [00:12,  1.83it/s]
34it [00:12,  1.95it/s]
35it [00:13,  2.16it/s]
36it [00:13,  2.20it/s]
37it [00:14,  2.24it/s]
38it [00:14,  2.46it/s]
39it [00:14,  2.61it/s

In [ ]:
# Realizar la predicción
prediccion = modelo.predict(df_clean)

In [ ]:
prediccion

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
# Añadimos las predcciones para visualizar los resultados
resultados['PREDICCIONES'] = prediccion
print(resultados)

                                   CONTENIDO A ANALIZAR  PREDICCIONES
0     Madrugo a trabajar por la causa Medellín. Le e...           0.0
1     Gracias a Medellín. Gracias a tantas personas ...           0.0
2     Soy Luis Bernardo Vélez, quiero contarles de d...           0.0
3     Soy Luis Bernardo Vélez, quiero contarles de d...           0.0
4     Nos han invisibilizado, han vulnerado nuestros...           0.0
...                                                 ...           ...
1796  Medellín necesita nuevos liderazgos. Necesitam...           0.0
1797  Este 29 de octubre Votamos #CarlosBallesterosA...           0.0
1798  #PorFinUnaMedellínDeTodxs | Cada vez somos más...           0.0
1799  #Medellín| Nuestra visión de la relación entre...           0.0
1800  Este 29 de octubre Votamos #CarlosBallesterosA...           0.0

[1801 rows x 2 columns]


In [ ]:
# recargamos el df original
#df_clean = pd.read_csv('/content/drive/MyDrive/Proyecto grado/Codigo/df_clean_g.csv')
df_clean = pd.read_csv('/content/drive/MyDrive/Proyecto grado/Codigo/df_clean_d2 - adcreative.csv')

In [ ]:
#agregamos la columna de preddciones al df original
df_clean['PREDICCIONES'] = resultados['PREDICCIONES']

In [ ]:
# exportamos el df
df_clean.to_csv('df_clean_d2_adcreative_hs.csv')

### Clasificacion de los discursos de odio

Del mismo autor del modelo anterior, usaremos su clasificador para categorizar los discursos encontrados

Repositorio del clasificador: https://github.com/esaidh266/Algorithm-for-classifying-hate-expressions-by-type-in-Spanish

In [ ]:
# nos movemos al directorio raiz
%cd /content/drive/MyDrive/Proyecto\ grado/Codigo

/content/drive/MyDrive/Proyecto grado/Codigo


In [ ]:
!git clone https://github.com/esaidh266/Algorithm-for-classifying-hate-expressions-by-type-in-Spanish.git

Cloning into 'Algorithm-for-classifying-hate-expressions-by-type-in-Spanish'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (63/63), done.
remote: Total 65 (delta 30), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (65/65), 5.64 MiB | 11.58 MiB/s, done.
Resolving deltas: 100% (30/30), done.


In [ ]:
!cd Algorithm-for-classifying-hate-expressions-by-type-in-Spanish/

/content/drive/MyDrive/Proyecto grado/Codigo/Algorithm-for-classifying-hate-expressions-by-type-in-Spanish


### importacion de librerias


In [ ]:
import torch
from huggingface_hub import login
from transformers import AutoModelForSequenceClassification, AutoTokenizer

login("") # token de acceso de huggingface

### Funcion y configuracion de variables

In [ ]:
# set rutas
MODEL_PATH = "/content/drive/MyDrive/Proyecto grado/Codigo/Algorithm-for-classifying-hate-expressions-by-type-in-Spanish/hate_type_model"
TOKENIZER = 'pysentimiento/robertuito-base-uncased'
LABEL_MAPPING = {
        0: 'general',
        1: 'político',
        2: 'misoginia',
        3: 'xenofobia',
        4: 'sexual'
    }

# funcion de deteccion
def _model_prediction(model, tokenizer, text):
    # Tokenize example text
    tokens = tokenizer(text, return_tensors="pt")

    # Predict
    with torch.no_grad():
        outputs = model(**tokens)

    # Obtain each class probability
    probabilidades = torch.softmax(outputs.logits, dim=1).squeeze()

    # Obtain the higher class
    clase_predicha = torch.argmax(probabilidades).item()

    # Return prediction
    return LABEL_MAPPING[clase_predicha]

In [ ]:
# creamos una nueva columna para la clasificacion
df_clean['tipo_hate_speech'] = ''

In [ ]:
# Load model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER)

for index, row in df_clean.iterrows():

  hate_pred = row.PREDICCIONES

  if hate_pred == 1.0:
    try:
      prediction = _model_prediction(model, tokenizer, row.transcripcion)
      df_clean.at[index, 'tipo_hate_speech'] = prediction
    except:
      continue

In [ ]:
df_clean.to_csv('df_clean_g_p_h.csv')

# Analisis de Sentimientos

Para el analisis de sentimientos de cada transcripcion usaremos la libreria pysentimiento, un toolkit multilingual para extracción de opiniones y análisis de sentimientos.

La libreria procesa y analiza:
- Sentimientos
- Emociones
- Hate Spech

Más informacion sobre la libreria: https://github.com/pysentimiento/pysentimiento?tab=readme-ov-file

In [ ]:
# instalacion de la libreria
!pip install pysentimiento

In [ ]:
!pip uninstall transformers -y
!pip install transformers


Found existing installation: transformers 4.49.0
Uninstalling transformers-4.49.0:
  Successfully uninstalled transformers-4.49.0
  Using cached transformers-4.49.0-py3-none-any.whl.metadata (44 kB)
Using cached transformers-4.49.0-py3-none-any.whl (10.0 MB)


In [ ]:
# Importacion de la libreria
from pysentimiento import create_analyzer

/usr/local/lib/python3.11/dist-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.0) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [ ]:
# carga del ultimo df guardado
df_clean = pd.read_csv('/content/drive/MyDrive/Proyecto grado/Codigo/df_clean_g_p_h_s_e_h.csv')

In [ ]:
# creamos dos columnas para almacenar el sentimineto y la probabilidad
df_clean['sentimiento'] = ''
df_clean['prob_sentimiento'] = ''

In [ ]:
analyzer = create_analyzer(task="sentiment", lang="es")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [ ]:
for index, row in df_clean.iterrows():

  trans_ = row.transcripcion

  sentimiento = analyzer.predict(trans_)

  df_clean.at[index, 'sentimiento'] = sentimiento.output
  df_clean.at[index, 'prob_sentimiento'] = sentimiento.probas

In [ ]:
df_clean.to_csv('df_clean_g_p_h_s_e_h.csv')

# Analisis de Emociones

In [ ]:
# creamos dos columna para almacenar la emocion y su probabilidad correspondiente
df_clean['emocion'] = ''
df_clean['prob_emocion'] = ''

In [ ]:
emotion_analyzer = create_analyzer(task="emotion", lang="es")

In [ ]:
for index, row in df_clean.iterrows():

  trans_ = row.transcripcion
  emocion = emotion_analyzer.predict(trans_)

  df_clean.at[index, 'emocion'] = emocion.output
  df_clean.at[index, 'prob_emocion'] = emocion.probas


In [ ]:
df_clean.to_csv('df_clean_g_p_h_s_e.csv')

#Menciones

In [ ]:
# carga del ultimo df guardado
df_clean = pd.read_csv('/content/drive/MyDrive/Proyecto grado/Codigo/df_clean_g_p_h_s_e_h.csv')

Creacion de diccionario de terminos relevantes a buscar

In [ ]:
candidatos_alcaldía =[
    "gilberto tobon",
    "carlos ballesteros",
    "jaime mejia",
    "juan camilo restrepo",
    "luis bernardo velez",
    "fico",
    "juan david valderrama",
    "juan carlos upegui",
    "deicy elena bermudez",
    "rodolfo correa",
    "maria paulina aguinaga",
    "liliana rendon",
    "albert corredor",
    "yordano",
    "felipe velez",
    "federico gutierrez",
    "daniel quintero"
]

ultimos_presidentes =[
    "gustavo petro",
    "ivan duque",
    "juan manuel santos",
    "alvaro uribe velez",
    "andres pastrana"
]

partidos_politicos = [
    "partido liberal colombiano",
    "partido conservador colombiano",
    "partido cambio radical",
    "partido alianza verde",
    "movimiento autoridades indigena de colombia",
    "aico",
    "partido alianza social independiente",
    "partido politico mira",
    "partido de la union por la gente",
    "partido de la u"
    "partido polo democratico alternativo",
    "partido politico union patriotica",
    "up"
    "partido centro democratico",
    "movimiento alternativo indigena y social",
    "mais",
    "partido comunes",
    "partido colombia justa libres",
    "partido colombia renaciente",
    "movimiento alianza democratica amplia 2021",
    "partido politico dignidad y compromiso",
    "movimiento politico colombia humana",
    "partido nuevo liberalismo",
    "movimiento salvacion nacional",
    "partido verde oxigeno",
    "partido comunista colombiano",
    "partido liga gobernantes anticorrupcion",
    "liga",
    "partido democrata colombiano",
    "partido ecologista colombiano",
    "partido politico la fuerza de la paz",
    "partido politico esperanza democratica",
    "partido politico gente en movimiento",
    "partido del trabajo de colombia",
    "movimiento politico poder popular",
    "movimiento politico soy porque somos"
]

In [ ]:
dic_candidatos_presidentes = candidatos_alcaldía + ultimos_presidentes

In [ ]:
# nueva columna para almacenar los terminos encontrados
df_clean['dic_candi_presid'] = ''
df_clean['dic_partidos'] = ''

In [ ]:
palabras_baneadas = ['de',
                     'la',
                     'por',
                     'y',
                     'en',
                     'trabajo',
                     'del',
                     'popular',
                     'porque',
                     'partido',
                     'cambio',
                     'poder',
                     'gente',
                     'social',
                     'paz',
                     'amplia',
                     'nacional',
                     'fuerza',
                     'esperanza',
                     'colombia',
                     'colombiano',
                     'politico',
                     'mira',
                     'asi'
                     ]

In [ ]:
for index, row in df_clean.iterrows():

  trans_ = row.trans_clean

  cand_presid = []
  partidos = []

  for i in dic_candidatos_presidentes:
    if i in trans_:
      cand_presid.append(i)
    else:
      split_terms = i.split()
      for j in split_terms:
        if j in cand_presid:
          continue
        if j in trans_:
          cand_presid.append(j)


  for i in partidos_politicos:
    if i in trans_:
      partidos.append(i)
    else:
      split_terms = i.split()
      for j in split_terms:
        if j in palabras_baneadas:
          continue
        if j in partidos:
          continue
        if j in trans_:
          partidos.append(j)

  df_clean.at[index, 'dic_candi_presid'] = cand_presid
  df_clean.at[index, 'dic_partidos'] = partidos

In [ ]:
df_clean.to_csv('df_clean_g_p_h_s_e_h_d.csv')